# Merging and Joining in Pandas

## Overview

**Merging/Joining** = Combining multiple DataFrames based on common columns or indices

### Why Merge/Join?

Real-world data is often **split across multiple tables**:
- Customer info in one table
- Order details in another
- Product catalog in a third

**We need to combine them for analysis!**

### Visual Example

```
Orders Table:           Customer Table:
order_id | customer_id  customer_id | name
1001     | C1           C1          | Alice
1002     | C2           C2          | Bob
1003     | C1           C3          | Charlie

        ↓ MERGE on customer_id ↓

Combined Table:
order_id | customer_id | name
1001     | C1          | Alice
1002     | C2          | Bob
1003     | C1          | Alice
```

### Three Main Methods

| Method | Purpose | SQL Equivalent |
|--------|---------|----------------|
| **`merge()`** | Join on columns | `JOIN` |
| **`join()`** | Join on index | `JOIN` (on index) |
| **`concat()`** | Stack DataFrames | `UNION` |

### Types of Joins

| Join Type | Description | Keeps |
|-----------|-------------|-------|
| **inner** | Only matching rows | Intersection |
| **left** | All from left + matching from right | Left table |
| **right** | All from right + matching from left | Right table |
| **outer** | All rows from both | Union |

### What We'll Learn
1. ✅ Inner, left, right, outer joins
2. ✅ merge() with different parameters
3. ✅ Joining on single/multiple columns
4. ✅ Handling duplicate keys
5. ✅ Merge indicators
6. ✅ concat() for stacking
7. ✅ Real-world database operations
8. ✅ Best practices

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.precision', 2)

print("✅ Libraries imported")
print(f"Pandas version: {pd.__version__}")

✅ Libraries imported
Pandas version: 2.2.3


## Sample Dataset: E-commerce System

We'll simulate a realistic e-commerce database with multiple related tables:

### Database Schema

```
┌─────────────┐     ┌──────────────┐     ┌──────────────┐
│  CUSTOMERS  │     │    ORDERS    │     │   PRODUCTS   │
├─────────────┤     ├──────────────┤     ├──────────────┤
│customer_id *│◄────│customer_id   │     │product_id *  │
│name         │     │order_id *    │     │product_name  │
│email        │     │product_id    │────►│category      │
│city         │     │quantity      │     │price         │
│signup_date  │     │order_date    │     │brand         │
└─────────────┘     └──────────────┘     └──────────────┘
       │                                         │
       └───────────────┐   ┌──────────────────┘
                       ▼   ▼
               ┌─────────────────┐
               │    REVIEWS      │
               ├─────────────────┤
               │review_id *      │
               │customer_id      │
               │product_id       │
               │rating (1-5)     │
               │review_text      │
               └─────────────────┘
```

This mimics a real database with **foreign key relationships**.

In [2]:
print("Creating sample e-commerce database...\n")

# Table 1: Customers
customers = pd.DataFrame({
    'customer_id': ['C001', 'C002', 'C003', 'C004', 'C005', 'C006'],
    'name': ['Alice Johnson', 'Bob Smith', 'Charlie Brown', 'Diana Prince', 'Eve Wilson', 'Frank Miller'],
    'email': ['alice@email.com', 'bob@email.com', 'charlie@email.com', 
              'diana@email.com', 'eve@email.com', 'frank@email.com'],
    'city': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia'],
    'signup_date': pd.date_range('2023-01-01', periods=6, freq='M')
})

# Table 2: Products
products = pd.DataFrame({
    'product_id': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'product_name': ['Laptop', 'Smartphone', 'Tablet', 'Headphones', 'Smartwatch'],
    'category': ['Electronics', 'Electronics', 'Electronics', 'Audio', 'Wearable'],
    'price': [1200, 800, 500, 150, 300],
    'brand': ['Dell', 'Apple', 'Samsung', 'Sony', 'Apple']
})

# Table 3: Orders
orders = pd.DataFrame({
    'order_id': ['O001', 'O002', 'O003', 'O004', 'O005', 'O006', 'O007', 'O008'],
    'customer_id': ['C001', 'C002', 'C001', 'C003', 'C002', 'C004', 'C001', 'C005'],
    'product_id': ['P001', 'P002', 'P003', 'P001', 'P004', 'P005', 'P002', 'P003'],
    'quantity': [1, 2, 1, 1, 3, 1, 1, 2],
    'order_date': pd.date_range('2024-01-01', periods=8, freq='3D')
})

# Table 4: Reviews
reviews = pd.DataFrame({
    'review_id': ['R001', 'R002', 'R003', 'R004', 'R005'],
    'customer_id': ['C001', 'C002', 'C001', 'C003', 'C007'],  # C007 doesn't exist!
    'product_id': ['P001', 'P002', 'P003', 'P001', 'P006'],   # P006 doesn't exist!
    'rating': [5, 4, 5, 3, 4],
    'review_text': ['Excellent!', 'Good value', 'Love it', 'Decent', 'Nice product']
})

print("CUSTOMERS TABLE:")
print(customers)
print(f"\nShape: {customers.shape}\n")

print("PRODUCTS TABLE:")
print(products)
print(f"\nShape: {products.shape}\n")

print("ORDERS TABLE:")
print(orders)
print(f"\nShape: {orders.shape}\n")

print("REVIEWS TABLE:")
print(reviews)
print(f"\nShape: {reviews.shape}")
print("\nNote: Reviews table has orphan records (C007, P006 don't exist!)")

Creating sample e-commerce database...

CUSTOMERS TABLE:
  customer_id           name              email          city signup_date
0        C001  Alice Johnson    alice@email.com      New York  2023-01-31
1        C002      Bob Smith      bob@email.com   Los Angeles  2023-02-28
2        C003  Charlie Brown  charlie@email.com       Chicago  2023-03-31
3        C004   Diana Prince    diana@email.com       Houston  2023-04-30
4        C005     Eve Wilson      eve@email.com       Phoenix  2023-05-31
5        C006   Frank Miller    frank@email.com  Philadelphia  2023-06-30

Shape: (6, 5)

PRODUCTS TABLE:
  product_id product_name     category  price    brand
0       P001       Laptop  Electronics   1200     Dell
1       P002   Smartphone  Electronics    800    Apple
2       P003       Tablet  Electronics    500  Samsung
3       P004   Headphones        Audio    150     Sony
4       P005   Smartwatch     Wearable    300    Apple

Shape: (5, 5)

ORDERS TABLE:
  order_id customer_id product_id

/var/folders/d1/sfvddbbx4tg14yxq7trqmhyw0000gn/T/ipykernel_10077/1260983053.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'signup_date': pd.date_range('2023-01-01', periods=6, freq='M')


## 1. Understanding Join Types

### Visual Representation

```
Left Table (A):      Right Table (B):
key | value_a        key | value_b
1   | a1             2   | b2
2   | a2             3   | b3
3   | a3             4   | b4

═══════════════════════════════════════════════════════════

INNER JOIN (how='inner'):  Only matching keys
key | value_a | value_b
2   | a2      | b2
3   | a3      | b3
Result: 2 rows

═══════════════════════════════════════════════════════════

LEFT JOIN (how='left'):  All from left + matching from right
key | value_a | value_b
1   | a1      | NaN      ← No match in right
2   | a2      | b2
3   | a3      | b3
Result: 3 rows (all from left)

═══════════════════════════════════════════════════════════

RIGHT JOIN (how='right'):  All from right + matching from left
key | value_a | value_b
2   | a2      | b2
3   | a3      | b3
4   | NaN     | b4       ← No match in left
Result: 3 rows (all from right)

═══════════════════════════════════════════════════════════

OUTER JOIN (how='outer'):  All from both
key | value_a | value_b
1   | a1      | NaN
2   | a2      | b2
3   | a3      | b3
4   | NaN     | b4
Result: 4 rows (union)
```

### When to Use Each Type

| Join Type | Use Case | Example |
|-----------|----------|----------|
| **inner** | Only complete records | Orders with valid customer info |
| **left** | Keep all from main table | All orders, even if product deleted |
| **right** | Keep all from reference table | All products, even if no orders |
| **outer** | Analysis of missing data | Find orphan records |

### Syntax

```python
# Basic merge
pd.merge(left_df, right_df, on='key', how='inner')

# Parameters:
# - on: column name(s) to join on
# - how: 'inner', 'left', 'right', 'outer'
# - left_on, right_on: different column names
# - suffixes: ('_left', '_right') for duplicate columns
```

## 2. Inner Join

**Inner Join** = Keep only rows where **key exists in both** tables

### Characteristics
- ✅ Returns only matching rows
- ✅ No NaN values from the join
- ✅ Smallest result set
- ❌ Loses non-matching data

### Use When
- You only want complete records
- Both tables must have matching keys
- Filtering out invalid references

In [3]:
print("=== INNER JOIN EXAMPLES ===\n")

# Example 1: Orders + Customers (inner join)
print("Example 1: Orders with customer details (INNER JOIN)")
orders_customers = pd.merge(orders, customers, on='customer_id', how='inner')
print(orders_customers[['order_id', 'customer_id', 'name', 'city', 'product_id', 'quantity']])
print(f"\nOriginal orders: {len(orders)}")
print(f"After inner join: {len(orders_customers)}")
print("All orders matched (all customers exist)\n")

# Example 2: Orders + Products
print("Example 2: Orders with product details (INNER JOIN)")
orders_products = pd.merge(orders, products, on='product_id', how='inner')
print(orders_products[['order_id', 'product_id', 'product_name', 'price', 'quantity']])
print(f"\nOriginal orders: {len(orders)}")
print(f"After inner join: {len(orders_products)}")
print()

# Example 3: Reviews + Customers (inner join shows data loss)
print("Example 3: Reviews with customer info (INNER JOIN)")
reviews_customers = pd.merge(reviews, customers, on='customer_id', how='inner')
print(reviews_customers[['review_id', 'customer_id', 'name', 'rating', 'review_text']])
print(f"\nOriginal reviews: {len(reviews)}")
print(f"After inner join: {len(reviews_customers)}")
print("⚠️ Lost 1 review (C007 doesn't exist in customers)\n")

# Example 4: Complete order details (3-way join)
print("Example 4: Complete order information (3-table inner join)")
# Step 1: Orders + Customers
temp = pd.merge(orders, customers, on='customer_id', how='inner')
# Step 2: Result + Products
complete_orders = pd.merge(temp, products, on='product_id', how='inner')
# Calculate total
complete_orders['total_amount'] = complete_orders['quantity'] * complete_orders['price']
print(complete_orders[['order_id', 'name', 'product_name', 'quantity', 'price', 'total_amount']])
print(f"\nTotal orders: {len(complete_orders)}")
print(f"Total revenue: ${complete_orders['total_amount'].sum():,.2f}")

=== INNER JOIN EXAMPLES ===

Example 1: Orders with customer details (INNER JOIN)
  order_id customer_id           name         city product_id  quantity
0     O001        C001  Alice Johnson     New York       P001         1
1     O002        C002      Bob Smith  Los Angeles       P002         2
2     O003        C001  Alice Johnson     New York       P003         1
3     O004        C003  Charlie Brown      Chicago       P001         1
4     O005        C002      Bob Smith  Los Angeles       P004         3
5     O006        C004   Diana Prince      Houston       P005         1
6     O007        C001  Alice Johnson     New York       P002         1
7     O008        C005     Eve Wilson      Phoenix       P003         2

Original orders: 8
After inner join: 8
All orders matched (all customers exist)

Example 2: Orders with product details (INNER JOIN)
  order_id product_id product_name  price  quantity
0     O001       P001       Laptop   1200         1
1     O002       P002   Smartpho

## 3. Left Join

**Left Join** = Keep **all rows from left** table + matching from right

### Characteristics
- ✅ Preserves all left table rows
- ✅ NaN for non-matching right columns
- ✅ Most common join type
- ❌ May create NaN values

### Use When
- Left table is your "main" data
- You want to enrich with optional info
- Need to preserve all original records

### Visual Example
```
Orders (LEFT):        Products (RIGHT):
order_id | prod_id    prod_id | name
O1       | P1         P1      | Laptop
O2       | P2         P2      | Phone
O3       | P999       [P999 missing!]

After LEFT JOIN:
order_id | prod_id | name
O1       | P1      | Laptop
O2       | P2      | Phone
O3       | P999    | NaN      ← Kept order, NaN for missing product
```

In [4]:
print("=== LEFT JOIN EXAMPLES ===\n")

# Example 1: Keep all orders, add customer info
print("Example 1: All orders with customer details (LEFT JOIN)")
orders_left = pd.merge(orders, customers, on='customer_id', how='left')
print(orders_left[['order_id', 'customer_id', 'name', 'product_id']])
print(f"\nOrders preserved: {len(orders_left)}")
print("All orders kept (even if customer deleted)\n")

# Example 2: Keep all reviews, add customer names
print("Example 2: All reviews with customer info (LEFT JOIN)")
reviews_left = pd.merge(reviews, customers, on='customer_id', how='left')
print(reviews_left[['review_id', 'customer_id', 'name', 'rating', 'review_text']])
print(f"\nOriginal reviews: {len(reviews)}")
print(f"After left join: {len(reviews_left)}")
print("Note: Review R005 has NaN name (C007 doesn't exist)\n")

# Example 3: Identify orphan records
print("Example 3: Find reviews with invalid customer IDs")
orphan_reviews = reviews_left[reviews_left['name'].isna()]
print(orphan_reviews[['review_id', 'customer_id', 'name', 'rating']])
print(f"\nOrphan reviews found: {len(orphan_reviews)}")
print("These reviews reference non-existent customers!\n")

# Example 4: Keep all products, show order counts
print("Example 4: All products with order counts (LEFT JOIN)")
products_orders = pd.merge(products, orders, on='product_id', how='left')
order_counts = products_orders.groupby(['product_id', 'product_name']).size().reset_index(name='order_count')
print(order_counts)
print("\nNote: All 5 products shown, even if no orders")

# Example 5: Calculate total revenue per product (including zero)
print("\nExample 5: Revenue per product (LEFT JOIN)")
products_with_sales = pd.merge(products, orders, on='product_id', how='left')
products_with_sales['revenue'] = products_with_sales['price'] * products_with_sales['quantity'].fillna(0)
revenue_summary = products_with_sales.groupby(['product_id', 'product_name'])['revenue'].sum().reset_index()
print(revenue_summary.sort_values('revenue', ascending=False))
print("\nAll products listed, even those with no sales")

=== LEFT JOIN EXAMPLES ===

Example 1: All orders with customer details (LEFT JOIN)
  order_id customer_id           name product_id
0     O001        C001  Alice Johnson       P001
1     O002        C002      Bob Smith       P002
2     O003        C001  Alice Johnson       P003
3     O004        C003  Charlie Brown       P001
4     O005        C002      Bob Smith       P004
5     O006        C004   Diana Prince       P005
6     O007        C001  Alice Johnson       P002
7     O008        C005     Eve Wilson       P003

Orders preserved: 8
All orders kept (even if customer deleted)

Example 2: All reviews with customer info (LEFT JOIN)
  review_id customer_id           name  rating   review_text
0      R001        C001  Alice Johnson       5    Excellent!
1      R002        C002      Bob Smith       4    Good value
2      R003        C001  Alice Johnson       5       Love it
3      R004        C003  Charlie Brown       3        Decent
4      R005        C007            NaN       4  Nic

## 4. Right Join

**Right Join** = Keep **all rows from right** table + matching from left

### Characteristics
- ✅ Preserves all right table rows
- ✅ NaN for non-matching left columns
- ❌ Less common (can use left join instead)

### Tip
```python
# These are equivalent:
pd.merge(A, B, how='right')  # Keep all from B
pd.merge(B, A, how='left')   # Keep all from B (just swap order)
```

**Recommendation**: Use LEFT JOIN and swap table order instead of RIGHT JOIN for consistency.

In [5]:
print("=== RIGHT JOIN EXAMPLES ===\n")

# Example 1: Keep all customers, show order info
print("Example 1: All customers with order info (RIGHT JOIN)")
orders_right = pd.merge(orders, customers, on='customer_id', how='right')
print(orders_right[['customer_id', 'name', 'order_id', 'product_id']].sort_values('customer_id'))
print(f"\nCustomers preserved: {customers['customer_id'].nunique()}")
print(f"Rows in result: {len(orders_right)}")
print("Note: C006 (Frank) has no orders (NaN in order columns)\n")

# Example 2: Find customers with no orders
print("Example 2: Customers who haven't ordered (RIGHT JOIN)")
no_orders = orders_right[orders_right['order_id'].isna()]
print(no_orders[['customer_id', 'name', 'city', 'order_id']])
print(f"\nCustomers with no orders: {len(no_orders)}\n")

# Example 3: Compare LEFT vs RIGHT
print("Example 3: LEFT vs RIGHT comparison")
print("\nOrders LEFT JOIN Customers (keep all orders):")
left_result = pd.merge(orders, customers, on='customer_id', how='left')
print(f"Rows: {len(left_result)}")

print("\nOrders RIGHT JOIN Customers (keep all customers):")
right_result = pd.merge(orders, customers, on='customer_id', how='right')
print(f"Rows: {len(right_result)}")
print("\nRIGHT has more rows (includes customers with no orders)\n")

# Example 4: Same result with LEFT by swapping
print("Example 4: RIGHT JOIN = LEFT JOIN with swapped tables")
right_way = pd.merge(orders, customers, on='customer_id', how='right')
left_way = pd.merge(customers, orders, on='customer_id', how='left')
print(f"Right join rows: {len(right_way)}")
print(f"Left join (swapped) rows: {len(left_way)}")
print("Both produce the same result!")

=== RIGHT JOIN EXAMPLES ===

Example 1: All customers with order info (RIGHT JOIN)
  customer_id           name order_id product_id
0        C001  Alice Johnson     O001       P001
1        C001  Alice Johnson     O003       P003
2        C001  Alice Johnson     O007       P002
3        C002      Bob Smith     O002       P002
4        C002      Bob Smith     O005       P004
5        C003  Charlie Brown     O004       P001
6        C004   Diana Prince     O006       P005
7        C005     Eve Wilson     O008       P003
8        C006   Frank Miller      NaN        NaN

Customers preserved: 6
Rows in result: 9
Note: C006 (Frank) has no orders (NaN in order columns)

Example 2: Customers who haven't ordered (RIGHT JOIN)
  customer_id          name          city order_id
8        C006  Frank Miller  Philadelphia      NaN

Customers with no orders: 1

Example 3: LEFT vs RIGHT comparison

Orders LEFT JOIN Customers (keep all orders):
Rows: 8

Orders RIGHT JOIN Customers (keep all customers):


## 5. Outer Join (Full Join)

**Outer Join** = Keep **all rows from both** tables

### Characteristics
- ✅ No data loss
- ✅ Shows all relationships and gaps
- ✅ Largest result set
- ❌ Many NaN values possible

### Use When
- Data quality analysis
- Finding orphan records
- Complete audit trail
- Identifying mismatches

### Visual Example
```
Left:              Right:
key | val_a        key | val_b
1   | a1           2   | b2
2   | a2           3   | b3
                   4   | b4

After OUTER JOIN:
key | val_a | val_b
1   | a1    | NaN     ← Only in left
2   | a2    | b2      ← In both
3   | NaN   | b3      ← Only in right
4   | NaN   | b4      ← Only in right
```

In [6]:
print("=== OUTER JOIN EXAMPLES ===\n")

# Example 1: Complete view of orders and customers
print("Example 1: All orders AND all customers (OUTER JOIN)")
orders_customers_outer = pd.merge(orders, customers, on='customer_id', how='outer')
print(orders_customers_outer[['customer_id', 'name', 'order_id', 'product_id']].sort_values('customer_id'))
print(f"\nTotal rows: {len(orders_customers_outer)}")
print(f"Orders: {len(orders)}")
print(f"Customers: {len(customers)}")
print("Result includes customers with no orders (C006)\n")

# Example 2: Find all mismatches
print("Example 2: Data quality check - find orphan records")
reviews_full = pd.merge(reviews, customers, on='customer_id', how='outer', indicator=True)
print(reviews_full[['customer_id', 'name', 'review_id', 'rating', '_merge']])
print("\n_merge column shows where data came from:")
print(reviews_full['_merge'].value_counts())
print()

# Example 3: Find customers with no reviews
print("Example 3: Customers who haven't reviewed")
customers_no_review = reviews_full[reviews_full['review_id'].isna()]
print(customers_no_review[['customer_id', 'name', 'city']].head())
print(f"\nCustomers with no reviews: {len(customers_no_review)}\n")

# Example 4: Find reviews from non-existent customers
print("Example 4: Orphan reviews (invalid customer_id)")
orphan = reviews_full[reviews_full['name'].isna()]
print(orphan[['review_id', 'customer_id', 'rating', 'review_text']])
print("\n⚠️ These reviews need data cleanup!\n")

# Example 5: Complete product-order analysis
print("Example 5: All products + all orders (OUTER)")
products_orders_full = pd.merge(products, orders, on='product_id', how='outer')
print(f"\nTotal rows: {len(products_orders_full)}")
print("\nProducts never ordered:")
never_ordered = products_orders_full[products_orders_full['order_id'].isna()]
print(never_ordered[['product_id', 'product_name', 'price']])
print(f"\nCount: {len(never_ordered.drop_duplicates('product_id'))}")

=== OUTER JOIN EXAMPLES ===

Example 1: All orders AND all customers (OUTER JOIN)
  customer_id           name order_id product_id
0        C001  Alice Johnson     O001       P001
1        C001  Alice Johnson     O003       P003
2        C001  Alice Johnson     O007       P002
3        C002      Bob Smith     O002       P002
4        C002      Bob Smith     O005       P004
5        C003  Charlie Brown     O004       P001
6        C004   Diana Prince     O006       P005
7        C005     Eve Wilson     O008       P003
8        C006   Frank Miller      NaN        NaN

Total rows: 9
Orders: 8
Customers: 6
Result includes customers with no orders (C006)

Example 2: Data quality check - find orphan records
  customer_id           name review_id  rating      _merge
0        C001  Alice Johnson      R001     5.0        both
1        C001  Alice Johnson      R003     5.0        both
2        C002      Bob Smith      R002     4.0        both
3        C003  Charlie Brown      R004     3.0       

## 6. Merging on Different Column Names

### Problem
Sometimes the **join key has different names** in each table:

```
Orders:              Customers:
cust_id | amount     customer_id | name
C1      | 100        C1          | Alice
```

### Solution: left_on and right_on

```python
pd.merge(orders, customers, 
         left_on='cust_id',      # Column in left table
         right_on='customer_id', # Column in right table
         how='inner')
```

### Multiple Join Keys

```python
# Join on multiple columns
pd.merge(df1, df2, 
         on=['col1', 'col2'],  # Both columns must match
         how='inner')
```

### Handling Duplicate Column Names

```python
# Default: adds _x and _y suffixes
pd.merge(df1, df2, on='key')  # name_x, name_y

# Custom suffixes
pd.merge(df1, df2, on='key', suffixes=('_left', '_right'))
```

In [7]:
print("=== MERGING ON DIFFERENT COLUMNS ===\n")

# Create tables with different column names
orders_alt = orders.rename(columns={'customer_id': 'cust_id', 'product_id': 'prod_id'})

# Example 1: Different column names
print("Example 1: Merge with different column names")
merged = pd.merge(orders_alt, customers, 
                  left_on='cust_id', 
                  right_on='customer_id',
                  how='inner')
print(merged[['order_id', 'cust_id', 'customer_id', 'name']].head())
print("\nNote: Both cust_id and customer_id appear in result\n")

# Example 2: Drop duplicate key column
print("Example 2: Clean up duplicate key columns")
merged_clean = pd.merge(orders_alt, customers, 
                        left_on='cust_id', 
                        right_on='customer_id',
                        how='inner').drop('customer_id', axis=1)
print(merged_clean[['order_id', 'cust_id', 'name', 'city']].head())
print()

# Example 3: Custom suffixes for duplicate columns
print("Example 3: Custom suffixes for duplicate columns")
# Add 'email' column to both tables
orders_with_email = orders.copy()
orders_with_email['email'] = 'order_' + orders_with_email['order_id'] + '@system.com'
merged_suffix = pd.merge(orders_with_email, customers, 
                         on='customer_id',
                         suffixes=('_order', '_customer'))
print(merged_suffix[['order_id', 'customer_id', 'email_order', 'email_customer']].head())
print()

# Example 4: Multiple join keys
print("Example 4: Join on multiple columns")
# Create scenario with composite key
inventory = pd.DataFrame({
    'product_id': ['P001', 'P001', 'P002', 'P002'],
    'location': ['Warehouse A', 'Warehouse B', 'Warehouse A', 'Warehouse B'],
    'stock': [50, 30, 100, 75]
})

sales = pd.DataFrame({
    'product_id': ['P001', 'P001', 'P002'],
    'location': ['Warehouse A', 'Warehouse B', 'Warehouse A'],
    'units_sold': [10, 5, 20]
})

inventory_sales = pd.merge(inventory, sales, 
                           on=['product_id', 'location'],  # Join on BOTH columns
                           how='left')
print(inventory_sales)
print("\nJoined on product_id AND location (composite key)")

=== MERGING ON DIFFERENT COLUMNS ===

Example 1: Merge with different column names
  order_id cust_id customer_id           name
0     O001    C001        C001  Alice Johnson
1     O002    C002        C002      Bob Smith
2     O003    C001        C001  Alice Johnson
3     O004    C003        C003  Charlie Brown
4     O005    C002        C002      Bob Smith

Note: Both cust_id and customer_id appear in result

Example 2: Clean up duplicate key columns
  order_id cust_id           name         city
0     O001    C001  Alice Johnson     New York
1     O002    C002      Bob Smith  Los Angeles
2     O003    C001  Alice Johnson     New York
3     O004    C003  Charlie Brown      Chicago
4     O005    C002      Bob Smith  Los Angeles

Example 3: Custom suffixes for duplicate columns
  order_id customer_id            email_order     email_customer
0     O001        C001  order_O001@system.com    alice@email.com
1     O002        C002  order_O002@system.com      bob@email.com
2     O003        

## 7. Merge Indicator

### What is Merge Indicator?

The **`indicator=True`** parameter adds a special column showing **where each row came from**.

### Values in _merge Column

| Value | Meaning |
|-------|----------|
| **left_only** | Row only in left table |
| **right_only** | Row only in right table |
| **both** | Row matched in both tables |

### Syntax

```python
# Default column name: '_merge'
pd.merge(df1, df2, on='key', how='outer', indicator=True)

# Custom column name
pd.merge(df1, df2, on='key', how='outer', indicator='source')
```

### Use Cases
- ✅ Data quality checks
- ✅ Find orphan records
- ✅ Identify mismatches
- ✅ Audit data integrity
- ✅ Debug joins

### Visual Example
```
After OUTER JOIN with indicator=True:

key | value | _merge
1   | a1    | left_only   ← Only in left table
2   | a2    | both        ← In both tables
3   | NaN   | right_only  ← Only in right table
```

In [8]:
print("=== MERGE INDICATOR EXAMPLES ===\n")

# Example 1: Basic indicator usage
print("Example 1: Merge with indicator")
merged_ind = pd.merge(reviews, customers, on='customer_id', how='outer', indicator=True)
print(merged_ind[['customer_id', 'name', 'review_id', '_merge']])
print("\n_merge value counts:")
print(merged_ind['_merge'].value_counts())
print()

# Example 2: Find records only in left
print("Example 2: Reviews without valid customer (left_only)")
orphan_reviews = merged_ind[merged_ind['_merge'] == 'left_only']
print(orphan_reviews[['review_id', 'customer_id', 'rating', '_merge']])
print(f"\nOrphan reviews: {len(orphan_reviews)}")
print("These need to be fixed or deleted!\n")

# Example 3: Find records only in right
print("Example 3: Customers with no reviews (right_only)")
no_reviews = merged_ind[merged_ind['_merge'] == 'right_only']
print(no_reviews[['customer_id', 'name', 'city', '_merge']].head())
print(f"\nCustomers with no reviews: {len(no_reviews)}")
print("Marketing opportunity!\n")

# Example 4: Find complete records
print("Example 4: Valid reviews with customer data (both)")
valid_reviews = merged_ind[merged_ind['_merge'] == 'both']
print(valid_reviews[['review_id', 'customer_id', 'name', 'rating', '_merge']])
print(f"\nValid reviews: {len(valid_reviews)}\n")

# Example 5: Custom indicator name
print("Example 5: Custom indicator column name")
custom_ind = pd.merge(orders, products, on='product_id', 
                      how='outer', indicator='data_source')
print(custom_ind[['order_id', 'product_id', 'product_name', 'data_source']].head(10))
print()

# Example 6: Data quality report
print("Example 6: Comprehensive data quality report")
quality_check = pd.merge(orders, products, on='product_id', 
                         how='outer', indicator='status')
print("\nData Quality Summary:")
print(f"Valid orders (matched products): {(quality_check['status'] == 'both').sum()}")
print(f"Invalid orders (missing products): {(quality_check['status'] == 'left_only').sum()}")
print(f"Products never ordered: {(quality_check['status'] == 'right_only').sum()}")

# Clean up _merge column when done
print("\n💡 Tip: Drop _merge column after analysis")
print("df = df.drop('_merge', axis=1)")

=== MERGE INDICATOR EXAMPLES ===

Example 1: Merge with indicator
  customer_id           name review_id      _merge
0        C001  Alice Johnson      R001        both
1        C001  Alice Johnson      R003        both
2        C002      Bob Smith      R002        both
3        C003  Charlie Brown      R004        both
4        C004   Diana Prince       NaN  right_only
5        C005     Eve Wilson       NaN  right_only
6        C006   Frank Miller       NaN  right_only
7        C007            NaN      R005   left_only

_merge value counts:
_merge
both          4
right_only    3
left_only     1
Name: count, dtype: int64

Example 2: Reviews without valid customer (left_only)
  review_id customer_id  rating     _merge
7      R005        C007     4.0  left_only

Orphan reviews: 1
These need to be fixed or deleted!

Example 3: Customers with no reviews (right_only)
  customer_id          name          city      _merge
4        C004  Diana Prince       Houston  right_only
5        C005    E

## 8. Concat - Stacking DataFrames

### concat() vs merge()

| Operation | Purpose | Use Case |
|-----------|---------|----------|
| **merge()** | Join on keys | Combine related tables |
| **concat()** | Stack DataFrames | Combine similar data |

### Visual Comparison

```
MERGE (horizontal):
DF1:        DF2:         Result:
id | val1   id | val2    id | val1 | val2
1  | a      1  | x       1  | a    | x
2  | b      2  | y       2  | b    | y

CONCAT (vertical):
DF1:         DF2:         Result:
id | value   id | value   id | value
1  | a       3  | c       1  | a
2  | b       4  | d       2  | b
                          3  | c
                          4  | d
```

### Syntax

```python
# Vertical stacking (default)
pd.concat([df1, df2, df3], axis=0)

# Horizontal stacking
pd.concat([df1, df2], axis=1)

# Ignore original index
pd.concat([df1, df2], ignore_index=True)

# Add keys to identify source
pd.concat([df1, df2], keys=['Dataset1', 'Dataset2'])
```

### Common Use Cases
- Combine data from multiple files
- Append new records
- Stack monthly data
- Combine train/test sets

### Parameters

| Parameter | Description | Default |
|-----------|-------------|----------|
| **axis** | 0=vertical, 1=horizontal | 0 |
| **ignore_index** | Reset index | False |
| **keys** | Add hierarchical index | None |
| **join** | 'inner' or 'outer' | 'outer' |

In [9]:
print("=== CONCAT EXAMPLES ===\n")

# Create sample data for concatenation
jan_sales = pd.DataFrame({
    'product': ['Laptop', 'Phone', 'Tablet'],
    'sales': [100, 150, 80],
    'month': ['Jan', 'Jan', 'Jan']
})

feb_sales = pd.DataFrame({
    'product': ['Laptop', 'Phone', 'Tablet'],
    'sales': [120, 160, 90],
    'month': ['Feb', 'Feb', 'Feb']
})

mar_sales = pd.DataFrame({
    'product': ['Laptop', 'Phone', 'Tablet'],
    'sales': [110, 140, 85],
    'month': ['Mar', 'Mar', 'Mar']
})

# Example 1: Basic vertical concatenation
print("Example 1: Stack monthly sales (vertical concat)")
all_sales = pd.concat([jan_sales, feb_sales, mar_sales])
print(all_sales)
print(f"\nShape: {all_sales.shape}")
print("Note: Index is preserved (0,1,2 repeated)\n")

# Example 2: Reset index
print("Example 2: Concat with reset index")
all_sales_clean = pd.concat([jan_sales, feb_sales, mar_sales], ignore_index=True)
print(all_sales_clean)
print("\nIndex is now sequential\n")

# Example 3: Add keys to identify source
print("Example 3: Concat with keys (hierarchical index)")
sales_with_keys = pd.concat([jan_sales, feb_sales, mar_sales], 
                             keys=['January', 'February', 'March'])
print(sales_with_keys)
print("\nMultiIndex shows which month each row came from\n")

# Example 4: Horizontal concatenation
print("Example 4: Horizontal concatenation (side by side)")
region_north = pd.DataFrame({'product': ['Laptop', 'Phone'], 'north_sales': [100, 150]})
region_south = pd.DataFrame({'product': ['Laptop', 'Phone'], 'south_sales': [120, 160]})
horizontal = pd.concat([region_north, region_south], axis=1)
print(horizontal)
print("\nNote: Duplicates 'product' column\n")

# Example 5: Handle mismatched columns
print("Example 5: Concat with different columns")
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'B': [5, 6], 'C': [7, 8]})
concat_diff = pd.concat([df1, df2], ignore_index=True)
print(concat_diff)
print("\nMissing values filled with NaN\n")

# Example 6: Inner join (only common columns)
print("Example 6: Concat with inner join")
concat_inner = pd.concat([df1, df2], join='inner', ignore_index=True)
print(concat_inner)
print("\nOnly column B (common to both) is kept\n")

# Example 7: Real-world - combine quarterly data
print("Example 7: Combine quarterly reports")
q1 = orders.iloc[:2].copy()
q2 = orders.iloc[2:4].copy()
q3 = orders.iloc[4:6].copy()

q1['quarter'] = 'Q1'
q2['quarter'] = 'Q2'
q3['quarter'] = 'Q3'

yearly_report = pd.concat([q1, q2, q3], ignore_index=True)
print(yearly_report[['order_id', 'customer_id', 'product_id', 'quarter']])
print(f"\nCombined shape: {yearly_report.shape}")

=== CONCAT EXAMPLES ===

Example 1: Stack monthly sales (vertical concat)
  product  sales month
0  Laptop    100   Jan
1   Phone    150   Jan
2  Tablet     80   Jan
0  Laptop    120   Feb
1   Phone    160   Feb
2  Tablet     90   Feb
0  Laptop    110   Mar
1   Phone    140   Mar
2  Tablet     85   Mar

Shape: (9, 3)
Note: Index is preserved (0,1,2 repeated)

Example 2: Concat with reset index
  product  sales month
0  Laptop    100   Jan
1   Phone    150   Jan
2  Tablet     80   Jan
3  Laptop    120   Feb
4   Phone    160   Feb
5  Tablet     90   Feb
6  Laptop    110   Mar
7   Phone    140   Mar
8  Tablet     85   Mar

Index is now sequential

Example 3: Concat with keys (hierarchical index)
           product  sales month
January  0  Laptop    100   Jan
         1   Phone    150   Jan
         2  Tablet     80   Jan
February 0  Laptop    120   Feb
         1   Phone    160   Feb
         2  Tablet     90   Feb
March    0  Laptop    110   Mar
         1   Phone    140   Mar
         2

## 9. Advanced Merging Scenarios

### Scenario 1: One-to-Many Relationships

```
Customers (One):     Orders (Many):
cust_id | name       cust_id | order_id
C1      | Alice      C1      | O1
C2      | Bob        C1      | O2  ← Same customer
                     C1      | O3  ← Same customer
                     C2      | O4

After merge: Each order gets customer info
```

### Scenario 2: Many-to-Many Relationships

```
Students:            Courses:
student_id | name    student_id | course
S1         | Alice   S1         | Math
S2         | Bob     S1         | Physics
                     S2         | Math
                     S2         | Chemistry

Result: Cartesian product for matching keys
```

### Scenario 3: Self-Join

Join a table with itself (e.g., employee-manager relationship)

```python
# Employees table with manager_id
pd.merge(employees, employees,
         left_on='manager_id',
         right_on='employee_id',
         suffixes=('_emp', '_mgr'))
```

### Scenario 4: Multiple Sequential Joins

```python
# Chain multiple merges
result = (orders
    .merge(customers, on='customer_id')
    .merge(products, on='product_id')
    .merge(categories, on='category_id')
)
```

In [10]:
print("=== ADVANCED MERGING SCENARIOS ===\n")

# Example 1: One-to-Many merge
print("Example 1: One-to-Many (1 customer → many orders)")
one_to_many = pd.merge(orders, customers, on='customer_id', how='inner')
print(one_to_many[['customer_id', 'name', 'order_id', 'product_id']])
print(f"\nCustomers: {customers.shape[0]}")
print(f"Orders: {orders.shape[0]}")
print(f"Result rows: {one_to_many.shape[0]}")
print("Customer info is duplicated for each order\n")

# Example 2: Check for duplicate keys
print("Example 2: Verify relationship cardinality")
print(f"Unique customers in customers table: {customers['customer_id'].nunique()}")
print(f"Unique customers in orders table: {orders['customer_id'].nunique()}")
print(f"Customer C001 has {(orders['customer_id'] == 'C001').sum()} orders")
print()

# Example 3: Self-join (employees and managers)
print("Example 3: Self-join (employees and their managers)")
employees = pd.DataFrame({
    'emp_id': ['E1', 'E2', 'E3', 'E4'],
    'name': ['Alice', 'Bob', 'Charlie', 'David'],
    'manager_id': [None, 'E1', 'E1', 'E2']  # Alice is CEO
})
print("Employees table:")
print(employees)

emp_mgr = pd.merge(employees, employees,
                   left_on='manager_id',
                   right_on='emp_id',
                   how='left',
                   suffixes=('', '_manager'))
print("\nWith manager names:")
print(emp_mgr[['emp_id', 'name', 'name_manager']])
print()

# Example 4: Multiple sequential joins
print("Example 4: Chain multiple joins (orders → customers → products)")
complete = (orders
    .merge(customers, on='customer_id', how='inner')
    .merge(products, on='product_id', how='inner')
)
complete['total_amount'] = complete['quantity'] * complete['price']
print(complete[['order_id', 'name', 'city', 'product_name', 'quantity', 'price', 'total_amount']])
print()

# Example 5: Aggregation after merge
print("Example 5: Customer spending analysis (merge + groupby)")
customer_spending = (orders
    .merge(customers, on='customer_id')
    .merge(products, on='product_id')
)
customer_spending['amount'] = customer_spending['quantity'] * customer_spending['price']

spending_summary = customer_spending.groupby(['customer_id', 'name', 'city']).agg(
    total_spent=('amount', 'sum'),
    num_orders=('order_id', 'count'),
    avg_order=('amount', 'mean')
).round(2).sort_values('total_spent', ascending=False)

print(spending_summary)
print()

# Example 6: Complex filter after merge
print("Example 6: High-value orders in specific cities")
high_value = complete[complete['total_amount'] > 1000]
city_filter = high_value[high_value['city'].isin(['New York', 'Los Angeles'])]
print(city_filter[['order_id', 'name', 'city', 'product_name', 'total_amount']])
print(f"\nHigh-value orders in target cities: {len(city_filter)}")

=== ADVANCED MERGING SCENARIOS ===

Example 1: One-to-Many (1 customer → many orders)
  customer_id           name order_id product_id
0        C001  Alice Johnson     O001       P001
1        C002      Bob Smith     O002       P002
2        C001  Alice Johnson     O003       P003
3        C003  Charlie Brown     O004       P001
4        C002      Bob Smith     O005       P004
5        C004   Diana Prince     O006       P005
6        C001  Alice Johnson     O007       P002
7        C005     Eve Wilson     O008       P003

Customers: 6
Orders: 8
Result rows: 8
Customer info is duplicated for each order

Example 2: Verify relationship cardinality
Unique customers in customers table: 6
Unique customers in orders table: 5
Customer C001 has 3 orders

Example 3: Self-join (employees and their managers)
Employees table:
  emp_id     name manager_id
0     E1    Alice       None
1     E2      Bob         E1
2     E3  Charlie         E1
3     E4    David         E2

With manager names:
  emp_id 

## 10. The join() Method

### join() vs merge()

| Feature | merge() | join() |
|---------|---------|--------|
| **Join on** | Columns | Index (default) |
| **Flexibility** | High | Lower |
| **Syntax** | More verbose | Simpler |
| **Use when** | Join on columns | Join on index |

### Syntax

```python
# Join on index (default)
df1.join(df2, how='left')

# Join df1's column with df2's index
df1.join(df2, on='key_column')

# Equivalent operations:
df1.join(df2)  # Join on index
pd.merge(df1, df2, left_index=True, right_index=True)
```

### When to Use join()

- ✅ DataFrames have meaningful indices
- ✅ Quick index-based joins
- ✅ Simpler syntax for index joins

### When to Use merge()

- ✅ Join on columns (most common)
- ✅ More control over join behavior
- ✅ Different column names
- ✅ Multiple join keys

In [11]:
print("=== JOIN() METHOD EXAMPLES ===\n")

# Prepare data with index
customers_idx = customers.set_index('customer_id')
products_idx = products.set_index('product_id')

# Example 1: Basic join on index
print("Example 1: Join on index")
orders_cust = orders.set_index('customer_id')
joined = orders_cust.join(customers_idx[['name', 'city']], how='left')
print(joined.head())
print()

# Example 2: join() vs merge() comparison
print("Example 2: join() vs merge() - same result")
print("\nUsing join():")
result_join = orders_cust.join(customers_idx[['name', 'city']])
print(result_join[['order_id', 'name', 'city']].head())

print("\nUsing merge():")
result_merge = pd.merge(orders, customers[['customer_id', 'name', 'city']], 
                        on='customer_id')
print(result_merge[['order_id', 'name', 'city']].head())
print()

# Example 3: Join with column
print("Example 3: Join df1's column with df2's index")
# orders has customer_id as column, customers_idx has it as index
joined_col = orders.join(customers_idx[['name', 'city']], 
                         on='customer_id', 
                         how='left')
print(joined_col[['order_id', 'customer_id', 'name', 'city']].head())
print()

# Example 4: Join multiple DataFrames
print("Example 4: Join multiple DataFrames at once")
orders_idx = orders.set_index('order_id')
cust_subset = customers_idx[['name', 'city']]
prod_subset = products_idx[['product_name', 'price']]

# Requires setting appropriate indices first
orders_with_cust = orders.set_index('customer_id').join(cust_subset)
print(orders_with_cust[['order_id', 'name', 'city', 'product_id']].head())
print()

# Example 5: Why merge() is often preferred
print("Example 5: merge() is more flexible for column-based joins")
print("\nScenario: Join on columns (most common case)")
print("\nWith merge() (straightforward):")
print("pd.merge(orders, customers, on='customer_id')")
print("\nWith join() (requires index manipulation):")
print("orders.set_index('customer_id').join(customers.set_index('customer_id'))")
print("\n💡 Tip: Use merge() for most cases, join() when working with indices")

=== JOIN() METHOD EXAMPLES ===

Example 1: Join on index
            order_id product_id  quantity order_date           name  \
customer_id                                                           
C001            O001       P001         1 2024-01-01  Alice Johnson   
C002            O002       P002         2 2024-01-04      Bob Smith   
C001            O003       P003         1 2024-01-07  Alice Johnson   
C003            O004       P001         1 2024-01-10  Charlie Brown   
C002            O005       P004         3 2024-01-13      Bob Smith   

                    city  
customer_id               
C001            New York  
C002         Los Angeles  
C001            New York  
C003             Chicago  
C002         Los Angeles  

Example 2: join() vs merge() - same result

Using join():
            order_id           name         city
customer_id                                     
C001            O001  Alice Johnson     New York
C002            O002      Bob Smith  Los Angeles
C

## 11. Best Practices & Common Pitfalls

### Best Practices ✅

**1. Validate Data Before Merging**
```python
# Check for duplicates in key columns
print(df['key'].duplicated().sum())

# Verify no NaN in join keys
print(df['key'].isna().sum())
```

**2. Use Merge Indicator for Debugging**
```python
# Always use indicator for outer joins
result = pd.merge(df1, df2, on='key', how='outer', indicator=True)
print(result['_merge'].value_counts())
```

**3. Choose Appropriate Join Type**
```python
# ✅ Inner: Only complete records
pd.merge(orders, customers, on='id', how='inner')

# ✅ Left: Keep all from main table
pd.merge(orders, customers, on='id', how='left')

# ✅ Outer: Data quality analysis
pd.merge(orders, customers, on='id', how='outer', indicator=True)
```

**4. Validate After Merge**
```python
# Check shape changes
print(f"Before: {len(df1)}, After: {len(merged)}")

# Check for unexpected duplicates
assert merged['key'].nunique() == df1['key'].nunique()
```

**5. Use Explicit Column Names**
```python
# ✅ Clear and explicit
pd.merge(orders, customers, 
         left_on='cust_id', 
         right_on='customer_id',
         how='inner')

# ❌ Ambiguous
orders.merge(customers)
```

**6. Handle Suffix Conflicts**
```python
# ✅ Meaningful suffixes
pd.merge(df1, df2, on='key', suffixes=('_sales', '_target'))

# ❌ Default _x, _y is confusing
pd.merge(df1, df2, on='key')
```

### Common Pitfalls ❌

**1. Unexpected Row Multiplication**
```python
# ❌ Many-to-many creates Cartesian product
# If df1 has 3 rows with key=1
# And df2 has 2 rows with key=1
# Result has 3×2 = 6 rows for key=1!

# ✅ Check for duplicates first
assert df1['key'].duplicated().sum() == 0
assert df2['key'].duplicated().sum() == 0
```

**2. NaN in Join Keys**
```python
# ❌ NaN keys don't match
# NaN == NaN is False in Pandas!

# ✅ Handle NaN before merging
df = df.dropna(subset=['key'])
# or
df['key'] = df['key'].fillna('UNKNOWN')
```

**3. Wrong Join Type**
```python
# ❌ Using inner when you need left
result = pd.merge(orders, products, on='id', how='inner')
# Silently drops orders with deleted products!

# ✅ Use left to keep all orders
result = pd.merge(orders, products, on='id', how='left')
```

**4. Not Validating Results**
```python
# ❌ Merge without checking
result = pd.merge(df1, df2, on='key')

# ✅ Always validate
print(f"Input: {len(df1)}, Output: {len(result)}")
print(f"NaN created: {result.isna().sum().sum()}")
```

**5. Index Issues**
```python
# ❌ Forgetting to reset index
result = pd.merge(df1, df2, on='key')
# Index is messy after merge

# ✅ Reset if needed
result = pd.merge(df1, df2, on='key').reset_index(drop=True)
```

### Performance Tips 🚀

**1. Sort Before Large Merges**
```python
# Faster for large datasets
df1 = df1.sort_values('key')
df2 = df2.sort_values('key')
result = pd.merge(df1, df2, on='key')
```

**2. Use Categorical for Repeated Merges**
```python
df['key'] = df['key'].astype('category')
```

**3. Filter Before Merging**
```python
# ✅ Filter first
df1_filtered = df1[df1['date'] > '2024-01-01']
result = pd.merge(df1_filtered, df2, on='key')

# ❌ Merge then filter (slower)
result = pd.merge(df1, df2, on='key')
result = result[result['date'] > '2024-01-01']
```

## 12. Practice Exercises

Use the provided e-commerce dataset for these exercises.

### Beginner Level (1-5)

1. **Inner join orders and customers**
   - Get order details with customer names

2. **Count orders per customer**
   - Merge orders with customers, then group by name

3. **Add product names to orders**
   - Inner join orders with products

4. **Find customers who never ordered**
   - Right join orders with customers, filter NaN orders

5. **Calculate total order amounts**
   - Merge orders with products, multiply quantity × price

### Intermediate Level (6-10)

6. **Complete order report**
   - Join orders + customers + products in one result

7. **Find orphan reviews**
   - Outer join reviews with customers, use indicator

8. **Customer spending by city**
   - Merge orders + customers + products, group by city

9. **Products never ordered**
   - Left join products with orders, filter NaN

10. **Average rating per product**
    - Inner join reviews with products, calculate mean rating

### Advanced Level (11-15)

11. **Customer lifetime value**
    - Calculate total spending per customer with all details

12. **Product popularity score**
    - Combine order count + average rating

13. **Cross-reference validation**
    - Find all inconsistencies across tables using outer joins

14. **Concat quarterly data**
    - Split orders by date, add quarter column, concat back

15. **Customer segment analysis**
    - Merge all tables, create customer segments based on spending

### Challenge Problems (16-20)

16. **Complete data quality report**
    - Check all foreign key relationships for orphans

17. **Customer churn analysis**
    - Identify customers who ordered once vs repeat customers

18. **Product recommendation pairs**
    - Find products frequently bought by same customer

19. **Revenue attribution**
    - Calculate revenue by product, category, brand, customer city

20. **Time-based cohort analysis**
    - Group customers by signup month, track ordering behavior

In [12]:
print("=== PRACTICE EXERCISE SOLUTIONS ===\n")
print("Try solving exercises first, then check solutions!\n")

# Solution 1
print("Solution 1: Inner join orders and customers")
sol1 = pd.merge(orders, customers, on='customer_id', how='inner')
print(sol1[['order_id', 'customer_id', 'name', 'product_id']].head())
print()

# Solution 2
print("Solution 2: Count orders per customer")
sol2 = pd.merge(orders, customers, on='customer_id')
order_counts = sol2.groupby('name').size().sort_values(ascending=False)
print(order_counts)
print()

# Solution 5
print("Solution 5: Calculate total order amounts")
sol5 = pd.merge(orders, products, on='product_id')
sol5['total_amount'] = sol5['quantity'] * sol5['price']
print(sol5[['order_id', 'product_name', 'quantity', 'price', 'total_amount']].head())
print()

# Solution 6
print("Solution 6: Complete order report (3-table join)")
sol6 = (orders
    .merge(customers, on='customer_id')
    .merge(products, on='product_id')
)
sol6['total'] = sol6['quantity'] * sol6['price']
print(sol6[['order_id', 'name', 'city', 'product_name', 'quantity', 'price', 'total']])
print()

# Solution 7
print("Solution 7: Find orphan reviews")
sol7 = pd.merge(reviews, customers, on='customer_id', how='outer', indicator=True)
orphans = sol7[sol7['_merge'] == 'left_only']
print(orphans[['review_id', 'customer_id', 'rating', '_merge']])
print(f"Orphan reviews: {len(orphans)}\n")

# Solution 8
print("Solution 8: Customer spending by city")
sol8 = (orders
    .merge(customers, on='customer_id')
    .merge(products, on='product_id')
)
sol8['amount'] = sol8['quantity'] * sol8['price']
city_spending = sol8.groupby('city')['amount'].sum().sort_values(ascending=False)
print(city_spending)
print()

# Solution 11
print("Solution 11: Customer lifetime value")
sol11 = (orders
    .merge(customers, on='customer_id')
    .merge(products, on='product_id')
)
sol11['amount'] = sol11['quantity'] * sol11['price']
ltv = sol11.groupby(['customer_id', 'name']).agg(
    total_spent=('amount', 'sum'),
    num_orders=('order_id', 'count'),
    avg_order=('amount', 'mean')
).round(2).sort_values('total_spent', ascending=False)
print(ltv)
print()

# Solution 14
print("Solution 14: Concat quarterly data")
# Split by date
q1_data = orders[orders['order_date'] < '2024-02-01'].copy()
q2_data = orders[orders['order_date'] >= '2024-02-01'].copy()

q1_data['quarter'] = 'Q1'
q2_data['quarter'] = 'Q2'

combined = pd.concat([q1_data, q2_data], ignore_index=True)
print(combined[['order_id', 'order_date', 'quarter']].head(10))
print()

# Solution 19
print("Solution 19: Revenue attribution")
sol19 = (orders
    .merge(customers, on='customer_id')
    .merge(products, on='product_id')
)
sol19['revenue'] = sol19['quantity'] * sol19['price']
attribution = sol19.groupby(['category', 'brand', 'city'])['revenue'].sum().sort_values(ascending=False)
print(attribution.head(10))

print("\n" + "="*80)
print("Try solving the remaining exercises on your own!")
print("="*80)

=== PRACTICE EXERCISE SOLUTIONS ===

Try solving exercises first, then check solutions!

Solution 1: Inner join orders and customers
  order_id customer_id           name product_id
0     O001        C001  Alice Johnson       P001
1     O002        C002      Bob Smith       P002
2     O003        C001  Alice Johnson       P003
3     O004        C003  Charlie Brown       P001
4     O005        C002      Bob Smith       P004

Solution 2: Count orders per customer
name
Alice Johnson    3
Bob Smith        2
Charlie Brown    1
Diana Prince     1
Eve Wilson       1
dtype: int64

Solution 5: Calculate total order amounts
  order_id product_name  quantity  price  total_amount
0     O001       Laptop         1   1200          1200
1     O002   Smartphone         2    800          1600
2     O003       Tablet         1    500           500
3     O004       Laptop         1   1200          1200
4     O005   Headphones         3    150           450

Solution 6: Complete order report (3-table join

## Quick Reference Card

### Merge Syntax

```python
# Basic merge
pd.merge(left, right, on='key', how='inner')

# Different column names
pd.merge(left, right, left_on='id1', right_on='id2', how='left')

# Multiple keys
pd.merge(left, right, on=['key1', 'key2'], how='outer')

# With indicator
pd.merge(left, right, on='key', how='outer', indicator=True)

# Custom suffixes
pd.merge(left, right, on='key', suffixes=('_l', '_r'))
```

### Join Types

```python
how='inner'   # Only matching rows (intersection)
how='left'    # All from left + matching from right
how='right'   # All from right + matching from left
how='outer'   # All from both (union)
```

### Concat Syntax

```python
# Vertical stacking (default)
pd.concat([df1, df2, df3])

# Horizontal stacking
pd.concat([df1, df2], axis=1)

# Reset index
pd.concat([df1, df2], ignore_index=True)

# Add source identifier
pd.concat([df1, df2], keys=['Source1', 'Source2'])

# Only common columns
pd.concat([df1, df2], join='inner')
```

### Join Method

```python
# Join on index
df1.join(df2, how='left')

# Join column to index
df1.join(df2, on='key_column')
```

### Common Patterns

```python
# Pattern 1: Enrich main table
orders_enriched = pd.merge(orders, customers, on='id', how='left')

# Pattern 2: Find missing relationships
check = pd.merge(df1, df2, on='key', how='outer', indicator=True)
missing = check[check['_merge'] != 'both']

# Pattern 3: Multiple table join
result = (orders
    .merge(customers, on='customer_id')
    .merge(products, on='product_id')
)

# Pattern 4: Stack similar data
all_data = pd.concat([jan, feb, mar], ignore_index=True)

# Pattern 5: Self-join
pd.merge(employees, employees,
         left_on='manager_id',
         right_on='emp_id',
         suffixes=('', '_mgr'))
```

### Validation Checks

```python
# Before merging
print(df['key'].duplicated().sum())  # Check duplicates
print(df['key'].isna().sum())        # Check NaN

# After merging
print(f"Before: {len(df1)}, After: {len(result)}")
print(result['_merge'].value_counts())  # With indicator
print(result.isna().sum())              # Check NaN created
```

### Decision Tree

```
Need to combine DataFrames?
    │
    ├─ Similar structure (same columns)?
    │   └─ Use concat() for stacking
    │
    └─ Related tables (foreign keys)?
        │
        ├─ Join on columns?
        │   └─ Use merge()
        │
        └─ Join on index?
            └─ Use join() or merge() with index=True

Which join type?
    ├─ Only complete records → inner
    ├─ Keep all from main table → left
    ├─ Keep all from lookup table → right
    └─ Find all mismatches → outer + indicator
```

## Summary

### Key Concepts Mastered ✅

**1. Join Types**
- **Inner**: Only matching rows (safest, smallest)
- **Left**: All from left + matching from right (most common)
- **Right**: All from right + matching from left (rare)
- **Outer**: All from both (data quality checks)

**2. Merge Operations**
- Join on single column: `on='key'`
- Join on multiple columns: `on=['key1', 'key2']`
- Different column names: `left_on='id1', right_on='id2'`
- Merge indicator: `indicator=True`
- Custom suffixes: `suffixes=('_x', '_y')`

**3. Concatenation**
- Vertical stacking: `pd.concat([df1, df2])`
- Horizontal stacking: `pd.concat([df1, df2], axis=1)`
- Reset index: `ignore_index=True`
- Source tracking: `keys=['A', 'B']`

**4. Advanced Scenarios**
- One-to-many relationships
- Many-to-many relationships
- Self-joins
- Sequential multi-table joins
- Data quality validation

---

### Method Selection Guide

| Task | Method | Example |
|------|--------|----------|
| Join on columns | `merge()` | Orders + Customers |
| Join on index | `join()` | Indexed DataFrames |
| Stack rows | `concat(axis=0)` | Monthly files |
| Stack columns | `concat(axis=1)` | Add features |
| Find orphans | `merge()` + `indicator=True` | Data quality |

---

### Real-World Applications

**E-commerce**
- Orders + Customers + Products → Complete order details
- Find customers who never ordered
- Calculate customer lifetime value
- Product performance analysis

**Data Quality**
- Identify orphan records
- Validate foreign key relationships
- Find missing data
- Audit data integrity

**Analytics**
- Combine fact and dimension tables
- Create customer 360° view
- Multi-source data integration
- Time-series concatenation

---

### Common Patterns

**1. Enrich Main Table**
```python
enriched = pd.merge(transactions, customer_info, 
                    on='customer_id', how='left')
```

**2. Find Missing Data**
```python
audit = pd.merge(orders, products, on='product_id', 
                 how='outer', indicator=True)
missing = audit[audit['_merge'] != 'both']
```

**3. Multi-Table Analysis**
```python
complete = (orders
    .merge(customers, on='customer_id')
    .merge(products, on='product_id')
    .merge(categories, on='category_id')
)
```

**4. Combine Time Periods**
```python
all_months = pd.concat([jan, feb, mar], ignore_index=True)
```

---

### Best Practices Checklist

Before merging:
- ☑ Check for duplicate keys
- ☑ Verify no NaN in join columns
- ☑ Understand relationship cardinality
- ☑ Choose appropriate join type

During merge:
- ☑ Use explicit column names
- ☑ Use indicator for outer joins
- ☑ Set meaningful suffixes

After merging:
- ☑ Validate row count
- ☑ Check for unexpected NaN
- ☑ Verify key relationships
- ☑ Reset index if needed

---

### Next Steps

After mastering merge/join:
1. **Pivot Tables** - Reshape and aggregate
2. **Time Series** - Date-based merging
3. **Advanced Indexing** - MultiIndex operations
4. **Database Integration** - SQL queries in Pandas
5. **Performance** - Optimize large merges

---

### Remember

- 🎯 **Inner** for safety, **Left** for completeness, **Outer** for auditing
- 🔍 **Always validate** before and after merging
- 📊 Use **merge indicator** to track data sources
- ⚡ **Filter before merging** for better performance
- 🧹 **Clean data** (duplicates, NaN) before joining

---

**Happy Merging! 🐼🔗**